In [1]:
import numpy as np
import pandas as pd

In [11]:
train_df = pd.read_csv('train.csv')

In [7]:
train_df.head(3)

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,2,"Weisz, Mrs. Leopold (Mathilde Francoise Pede)",female,29.0,1,0,228414,26.00,NaN,S,1
1,3,"Williams, Mr. Howard Hugh ""Harry""",male,NaN,0,0,A/5 2466,8.05,NaN,S,0
2,2,"Morley, Mr. Henry Samuel (""Mr Henry Marshall"")",male,39.0,0,0,250655,26.00,NaN,S,0


In [12]:
train_df.drop(['Name', 'Ticket'], axis=1, inplace=True)

In [13]:
train_df.head(3)

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Survived
0,2,female,29.0,1,0,26.00,NaN,S,1
1,3,male,NaN,0,0,8.05,NaN,S,0
2,2,male,39.0,0,0,26.00,NaN,S,0


In [14]:
train_df.isnull().sum()

Pclass        0
Sex           0
Age         132
SibSp         0
Parch         0
Fare          0
Cabin       514
Embarked      1
Survived      0
dtype: int64

In [16]:
train_df['Age'].fillna(train_df['Age'].median(skipna=True), inplace=True)
train_df['Embarked'].fillna(train_df['Embarked'].value_counts().idxmax(), inplace=True)

In [17]:
train_df.isnull().sum()

Pclass        0
Sex           0
Age           0
SibSp         0
Parch         0
Fare          0
Cabin       514
Embarked      0
Survived      0
dtype: int64

In [18]:
train_df.drop('Cabin', axis=1, inplace=True)

In [19]:
train_df.head(3)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,2,female,29.0,1,0,26.00,S,1
1,3,male,29.0,0,0,8.05,S,0
2,2,male,39.0,0,0,26.00,S,0


In [20]:
train_df['TravelAlone'] = np.where(train_df['SibSp'] + train_df['Parch'] > 0, 0, 1)
train_df.drop(['SibSp', 'Parch'], axis=1, inplace=True)

In [21]:
train_df.head(2)

,Pclass,Sex,Age,Fare,Embarked,Survived,TravelAlone
0,2,female,29.0,26.00,S,1,0
1,3,male,29.0,8.05,S,0,1


In [22]:
train_df = pd.get_dummies(train_df, columns=['Pclass', 'Sex', 'Embarked'])

In [23]:
train_df.head(3)

,Age,Fare,Survived,TravelAlone,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,29.0,26.00,1,0,0,1,0,1,0,0,0,1
1,29.0,8.05,0,1,0,0,1,0,1,0,0,1
2,39.0,26.00,0,1,0,1,0,0,1,0,0,1


In [24]:
train_df.drop('Sex_female', axis=1, inplace=True)

In [25]:
train_df.head(2)

,Age,Fare,Survived,TravelAlone,Pclass_1,Pclass_2,Pclass_3,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,29.0,26.00,1,0,0,1,0,0,0,0,1
1,29.0,8.05,0,1,0,0,1,1,0,0,1


In [37]:
X = train_df.drop('Survived', axis=1).values


In [38]:
type(X)

numpy.ndarray

In [39]:
Y = train_df.Survived.values

In [40]:
type(Y)

numpy.ndarray

In [51]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=1)

In [52]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [53]:
X_train.shape

(534, 10)

In [54]:
X_test.shape

(134, 10)

In [56]:
train_ones = np.ones(len(X_train))
test_ones = np.ones(len(X_test))
train_ones = train_ones.reshape(-1, 1)
test_ones = test_ones.reshape(-1, 1)
X_train = np.append(X_train, train_ones, axis=1)
X_test = np.append(X_test, test_ones, axis=1)

In [66]:
X_train.shape, X_test.shape

((534, 11), (134, 11))

In [93]:

tm = np.zeros(11)
hx = sigmoid(X_train[1, :], tm)
hx

0.5

In [98]:
def sigmoid(x, m):
    z = np.dot(x, m)
    return 1.0 / (1 + np.exp(-z))

In [103]:
def cost(X_train, y_train, m):
    total_cost = 0
    M = len(X_train)
    n_cols = X_train.shape[1]
    
    for i in range(M):
        x = X_train[i, :]
        y = y_train[i]
        hx = sigmoid(x, m)
        
        for j in range(n_cols):
            total_cost += (-1 / M) * (y) * (np.log(hx)) - (1 - y) * np.log(1 - hx)
    
    return total_cost

In [60]:
def step_gradient(X_train, y_train, learning_rate, m):
    n_cols = X_train.shape[1]
    m_slope = np.zeros(n_cols)
    M = len(X_train)
    
    for i in range(M):
        x = X_train[i, :]
        y = y_train[i]
        hx = sigmoid(x, m)
        
        for j in range(n_cols):
            m_slope[j] += (-1/M) * (y - hx) * x[j]
        m = m - learning_rate * m_slope
    
    return m

In [96]:
def gd(X_train, y_train, learning_rate, num_iterations):
    n_cols = X_train.shape[1]
    m = np.zeros(n_cols)
    
    for i in range(num_iterations):
        m = step_gradient(X_train, y_train, learning_rate, m)
        print(i, " Cost: ", cost(X_train, y_train, m))
    return m

In [63]:
def run():
    learning_rate = 0.001
    num_iterations = 500
    m = gd(X_train, y_train, learning_rate, num_iterations)
    
    return m

In [104]:
m = run()

0  Cost:  2267.54754854
1  Cost:  2152.37524064
2  Cost:  2058.88709797
3  Cost:  1981.43009107
4  Cost:  1916.06737895
5  Cost:  1860.02814353
6  Cost:  1811.3290088
7  Cost:  1768.52266206
8  Cost:  1730.53197077
9  Cost:  1696.53976175
10  Cost:  1665.91465114
11  Cost:  1638.16037598
12  Cost:  1612.88062779
13  Cost:  1589.75425536
14  Cost:  1568.51750586
15  Cost:  1548.95111408
16  Cost:  1530.8707794
17  Cost:  1514.12004315
18  Cost:  1498.56488965
19  Cost:  1484.08960114
20  Cost:  1470.59353614
21  Cost:  1457.98859634
22  Cost:  1446.19721252
23  Cost:  1435.15072652
24  Cost:  1424.78807835
25  Cost:  1415.05473115
26  Cost:  1405.90178321
27  Cost:  1397.28522867
28  Cost:  1389.16533737
29  Cost:  1381.50613103
30  Cost:  1374.27493806
31  Cost:  1367.44201294
32  Cost:  1360.98020895
33  Cost:  1354.86469555
34  Cost:  1349.07271299
35  Cost:  1343.58335847
36  Cost:  1338.377399
37  Cost:  1333.43710708
38  Cost:  1328.74611577
39  Cost:  1324.28929068
40  Cost:  132

323  Cost:  1218.69283632
324  Cost:  1218.68577971
325  Cost:  1218.67881032
326  Cost:  1218.67192704
327  Cost:  1218.66512875
328  Cost:  1218.65841436
329  Cost:  1218.65178278
330  Cost:  1218.64523297
331  Cost:  1218.63876385
332  Cost:  1218.63237442
333  Cost:  1218.62606363
334  Cost:  1218.61983049
335  Cost:  1218.61367401
336  Cost:  1218.6075932
337  Cost:  1218.60158711
338  Cost:  1218.59565477
339  Cost:  1218.58979526
340  Cost:  1218.58400764
341  Cost:  1218.57829101
342  Cost:  1218.57264446
343  Cost:  1218.5670671
344  Cost:  1218.56155806
345  Cost:  1218.55611647
346  Cost:  1218.55074149
347  Cost:  1218.54543226
348  Cost:  1218.54018796
349  Cost:  1218.53500777
350  Cost:  1218.52989089
351  Cost:  1218.52483651
352  Cost:  1218.51984384
353  Cost:  1218.51491212
354  Cost:  1218.51004058
355  Cost:  1218.50522846
356  Cost:  1218.50047502
357  Cost:  1218.49577952
358  Cost:  1218.49114124
359  Cost:  1218.48655945
360  Cost:  1218.48203346
361  Cost:  12

m

In [108]:
def predict(X_test, m):
    M = len(X_test)
    y_pred = np.zeros(M)
    
    for i in range(M):
        x = X_test[i, :]
        p = sigmoid(x, m)
        if (p > 0.5):
            y_pred[i] = 1
        else:
            y_pred[i] = 0
    
    return y_pred

In [109]:
y_pred = predict(X_test, m)

In [110]:
y_pred

array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  1.,  1.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  1.,  1.,  0.,
        1.,  1.,  0.,  1.,  1.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  0.,
        1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,  1.,  1.,  0.,  0.,
        0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,  1.,  0.,  0.,  0.,
        1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,
        0.,  0.,  1.,  0.,  1.,  1.,  0.,  0.,  1.,  1.,  1.,  0.,  1.,
        1.,  0.,  1.,  0.,  1.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  1.,
        1.,  0.,  1.,  0.,  1.,  0.,  1.,  1.,  0.,  1.,  1.,  0.,  0.,
        0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  1.])

In [111]:
from sklearn.metrics import accuracy_score, confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [112]:
cm

array([[67, 16],
       [ 8, 43]])

In [113]:
accuracy_score(y_test, y_pred)

0.82089552238805974